# Logica cu predicate (3). Înlănțuire înainte în Sisteme bazate pe reguli

- Tudor Berariu
- Andrei Olaru

## 1. Introducere
### Scopul laboratorului

Scopul acestui laborator îl reprezintă înțelegerea și implementarea demonstrării teoremelor prin înlănțuire înainte.

### Clauze definite. Reguli. Fapte

În cadrul acestui laborator vom folosi un tip anume de formule, mai precis **clauze definite**. Acestea sunt clauze Horn cu exact un literal pozitiv.

$$\neg p_1 \vee \neg p_2 \vee \ldots \vee \neg p_N \vee c$$

În scrierea echivalentă:

$$\left( p_1 \wedge p_2 \wedge \ldots \wedge p_N \right) \rightarrow c$$

devine evident ce reprezintă clauzele definite. Ele au o formă asemănătore regulilor.

Vom numi **regulă** o clauză definită cu cel puțin un literal negativ și vom numi **fapt** o clauză definită cu un singur literal (cel pozitiv).

### Problema de rezolvat

Problema pe care o vom rezolva astăzi se enunță astfel: *dată fiind o bază de cunoștințe* `kb` *formată din clauze definite (fapte și reguli), să se demonstreze o teoremă* `t`.

## 2. Funcții utile din laboratoarele anterioare

### Cerința 0

Salvați rezolvarea laboratorului 5 (*Reprezentare și Unificare*) cu numele `Lab05.py`. Vom folosi și funcțiile deja implementate din Laboratorul 6. Din acesta, funcțiile importante pentru astăzi sunt:
 - `make_var`, `make_const`, `make_atom` - utile pentru a construi atomi. De exemplu, $$Vreme(maine, Frig)$$ se construiește astfel:
     
     `make_atom("Vreme", make_var("maine"), make_const("Frig"))`
 
 - `unify` și `substitute` - utile pentru calcularea celui mai general unificator pentru două formule și pentru aplicarea unei substituții
 - `is_positive_literal` și `is_negative_literal`
 - `add_statement` - adaugă o clauză definită unei baze de cunoștințe. Avem două variante de utilizare în acest laborator:
   * `add_statement(kb, Atom)` - adaugă în kb faptul Atom
   * `add_statement(kb, C, P1, P2, Pn)` - adaugă în kb regula $\left( p_1 \wedge p_2 \wedge p_N \right) \rightarrow c$ sub forma:
   
   `make_or(make_neg(P1), make_neg(P2), make_neg(Pn), C)`
   
### Cerința 1

Faceți următoarea modificare funcției `unify`: antetul funcției

    def unify(f1, f2):
        ...
        subst = {}

trebuie transformat astfel încât să primească un al treilea parametru, o substituție de pornire

    def unify(f1, f2, subst=None):
        if not subst:
            subst = {}

**Nu uitați** ca după modificarea lui `unify` să descărcați din nou laboratorul 5 ca fișier Python și să dați Kernel restart în acest fișier.

In [2]:
from Lab05 import make_var, make_const, make_atom, make_or, make_neg, \
                is_variable, is_constant, is_atom, is_function_call, \
                print_formula, get_args, get_head, get_name, get_value ,\
                unify, substitute
from Lab06Skel import add_statement, is_positive_literal, is_negative_literal, \
                make_unique_var_names, print_KB
from Lab05tester import *

## 3. Baza de cunoștințe

Să se completeze reprezentarea bazei de cunoștințe de mai jos știind că ea corespunde următoarelor afirmații:

[TODO 2.1:] *Dacă a plouat două zile la rând, a treia zi va fi frumos.* [TODO 2.2:] *Dacă a fost frumos trei zile la rând, în cea de-a patra zi va ploua.* [TODO 2.3:] *Un student merge întotdeauna la munte dacă este frumos într-o zi de weekend. Cine merge la munte și practică un sport de iarnă va avea activități legate de acel sport de iarnă.*

*Arsenie și Nectarie sunt studenți. Arsenie practică volei și schi, iar Nectarie practică schi și sanie. Voleiul este un sport de vară, în timp ce schiul și sania sunt sporturi de iarnă. Vineri plouă; luni, marți și miercuri este frumos*

### Cerința 2:

Completați mai jos clauzele definite pentru primele 3 propoziții.

In [3]:
def get_sports_kb():
    sports_kb = []
    # Predicatul 'Consecutive'
    add_statement(sports_kb, make_atom('Consecutive', make_const('Luni'), make_const('Marti')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Marti'), make_const('Miercuri')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Miercuri'), make_const('Joi')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Joi'), make_const('Vineri')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Vineri'), make_const('Sambata')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Sambata'), make_const('Duminica')))
    # Predicatul 'Weekend'
    add_statement(sports_kb, make_atom('Weekend', make_const('Sambata')))
    add_statement(sports_kb, make_atom('Weekend', make_const('Duminica')))
    # Predicatul 'Ploua'
    add_statement(sports_kb, make_atom('Ploua', make_const('Vineri'))) 
    # Predicatul 'Frumos'
    add_statement(sports_kb, make_atom('Frumos', make_const('Luni')))
    add_statement(sports_kb, make_atom('Frumos', make_const('Marti')))
    add_statement(sports_kb, make_atom('Frumos', make_const('Miercuri')))
    
    # TODO 2.1: Dacă a plouat două zile la rând, a treia zi va fi frumos.
    add_statement(sports_kb, make_atom('Frumos', make_var('z')), make_atom('Consecutive', make_var('x'), make_var('y')),
                  make_atom('Consecutive', make_var('y'), make_var('z')), make_atom('Ploua', make_var('x')),
                  make_atom('Ploua', make_var('y')))
    
    # TODO 2.2: Dacă a fost frumos trei zile la rând, în cea de-a patra zi va ploua.
    add_statement(sports_kb, make_atom('Ploua', make_var('d')), make_atom('Consecutive', make_var('a'), make_var('b')),
                  make_atom('Consecutive', make_var('b'), make_var('c')), make_atom('Consecutive', make_var('c'), make_var('d')),
                  make_atom('Frumos', make_var('a')), make_atom('Frumos', make_var('b')), make_atom('Frumos', make_var('c')))
    
    # Predicatul 'Student'
    add_statement(sports_kb, make_atom('Student', make_const('Nectarie')))
    add_statement(sports_kb, make_atom('Student', make_const('Arsenie')))
    # MergeLaMunte (cine, cand)
    # TODO 2.3: Un student merge întotdeauna la munte dacă este frumos într-o zi de weekend.
    add_statement(sports_kb, make_atom('MergeLaMunte', make_var('stud'), make_var('day')),
                  make_atom('Student', make_var('stud')), make_atom('Weekend', make_var('day')),
                  make_atom('Frumos', make_var('day')))
    
    # Predicatul 'SportDeVara'
    add_statement(sports_kb, make_atom('SportDeVara', make_const('Volei')))
    # Predicatul 'SportDeIarna'
    add_statement(sports_kb, make_atom('SportDeIarna', make_const('Schi')))
    add_statement(sports_kb, make_atom('SportDeIarna', make_const('Sanie')))
    # Predicatul 'PracticaSport'
    add_statement(sports_kb, make_atom('PracticaSport', make_const('Nectarie'), make_const('Schi')))
    add_statement(sports_kb, make_atom('PracticaSport', make_const('Nectarie'), make_const('Sanie')))
    add_statement(sports_kb, make_atom('PracticaSport', make_const('Arsenie'), make_const('Schi')))
    add_statement(sports_kb, make_atom('PracticaSport', make_const('Arsenie'), make_const('Volei')))
    # Predicatul 'Activitate'
    add_statement(sports_kb, make_atom('Activitate', make_var('who'), make_var('what'), make_var('when')),
                  make_atom('MergeLaMunte', make_var('who'), make_var('when')),
                  make_atom('PracticaSport', make_var('who'), make_var('what'))
                 )
    make_unique_var_names(sports_kb)
    return sports_kb


print("Baza de cunoștințe se prezintă astfel:")
skb = get_sports_kb()
print_KB(skb)
print("==================== \n Baza de cunoștințe arată intern astfel:")
print("" + "".join([(str(s) + "\n") for s in skb]))

Baza de cunoștințe se prezintă astfel:
Added statement Consecutive(Luni, Marti)
Added statement Consecutive(Marti, Miercuri)
Added statement Consecutive(Miercuri, Joi)
Added statement Consecutive(Joi, Vineri)
Added statement Consecutive(Vineri, Sambata)
Added statement Consecutive(Sambata, Duminica)
Added statement Weekend(Sambata)
Added statement Weekend(Duminica)
Added statement Ploua(Vineri)
Added statement Frumos(Luni)
Added statement Frumos(Marti)
Added statement Frumos(Miercuri)
Added statement (SAU NOTConsecutive(?x, ?y) NOTConsecutive(?y, ?z) NOTPloua(?x) NOTPloua(?y) Frumos(?z))
Added statement (SAU NOTConsecutive(?a, ?b) NOTConsecutive(?b, ?c) NOTConsecutive(?c, ?d) NOTFrumos(?a) NOTFrumos(?b) NOTFrumos(?c) Ploua(?d))
Added statement Student(Nectarie)
Added statement Student(Arsenie)
Added statement (SAU NOTStudent(?stud) NOTWeekend(?day) NOTFrumos(?day) MergeLaMunte(?stud, ?day))
Added statement SportDeVara(Volei)
Added statement SportDeIarna(Schi)
Added statement SportDeIar

## 4. Funcții auxiliare

**Cerința 3:** Implementați funcțiile `get_premises`, `get_conclusion`, `is_fact` și `is_rule`. Toate acestea primesc o clauză definită (în baza de cunoștințe dată, poate fi un atom singur sau o disjuncție de literali) și întorc ceea ce specifică numele lor.

In [4]:
def get_premises(formula):
    # TODO
    if is_atom(formula):
        return []
    res = []
    n = len(get_args(formula))
    for neg in get_args(formula)[0:n-1]:
        res = res + get_args(neg)
    return res

def get_conclusion(formula):
    # TODO
    #print(get_args(formula))
    n = len(get_args(formula))
    return get_args(formula)[n-1]

def is_fact(formula):
    # TODO
    if is_atom(formula):
        return True
    return False

def is_rule(formula):
    # TODO
    if len(get_args(formula)) > 1:
        return True
    return False

# Test!
# formula: P(x) ^ Q(x) -> R(x)
f = make_or(make_neg(make_atom("P", make_var("x"))), make_neg(make_atom("Q", make_var("x"))), make_atom("R", make_var("x")))
print("".join([(print_formula(p, True) + " ; ") for p in get_premises(f)])[:-3]) # Should be P(?x) ; Q(?x)
print_formula(get_conclusion(f)) # Should be R(?x)
print(is_rule(f)) # must be True
print(is_fact(get_conclusion(f))) # must be True

P(?x) ; Q(?x)
R(?x)
True
True


In [5]:
def equal_terms(args1, args2):
    if len(args1) != len(args2):
        # Predicatele au aritate diferita
        return False
    for i, arg in enumerate(args2):
        if is_constant(arg):
            if not is_constant(args1[i]) or get_value(args1[i]) != get_value(arg):
                return False
        if is_variable(arg):
            if not is_variable(args1[i]) or get_name(args1[i]) != get_name(arg):
                return False
        if is_function_call(arg):
            if not is_function(args1[i]) or get_head(args1[i]) != get_head(arg):
                return False
            if not equal_terms(get_args(args1[i]), get_args(arg)):
                return False
    return True

def is_equal_to(a1, a2):
    # Nu verifica funcții
    if not is_atom(a1):
        # a1 nu este atom
        return False
    if get_head(a1) != get_head(a2):
        # Predicatele au nume diferite
        return False
    return equal_terms(get_args(a1), get_args(a2))
    

## 5. Demonstrarea teoremelor prin înlănțuire înainte

**Cerința 4:**

Implementați funcția `apply_rule(rule, facts)` care primește o regulă și un set de fapte și întoarce toate faptele care pot fi determinate prin aplicarea regulii pe faptele date.

Folosiți-vă de `unify`, `substitute`, dar și de `get_premises` și `get_conclusion` implementate mai devreme.

In [18]:
from copy import deepcopy
# from __future__ import print_function

def apply_rule(rule, facts, i = 0):
    # TODO
    res = []
    premises = get_premises(rule)
    
    if i >= len(premises):
        return []
    
    for atom in facts:
        subst = unify(premises[i], atom)
        new_rule = deepcopy(rule)
        if subst != None and subst != False:
            new_rule = substitute(new_rule, subst)
            if i < len(premises) - 1:
                res = res + apply_rule(new_rule, facts, i + 1)
            else:
                res = res + [get_conclusion(new_rule)]
    
    return res

# Test!
# Rule: P(x) => Q(x)
# Facts: P(1)
for f in apply_rule( 
        make_or(make_neg(make_atom("P", make_var("x"))), make_atom("Q", make_var("x"))), [make_atom("P", make_const(1))]):
    print_formula(f) # should be Q(1)
print("=====")
# Rule: P(x) ^ Q(x) => R(x)
# Facts: P(1), P(2), P(3), Q(3), Q(2)
for f in apply_rule( 
        make_or(
            make_neg(make_atom("P", make_var("x"))),
            make_neg(make_atom("Q", make_var("x"))),
            make_atom("R", make_var("x"))),
        [make_atom("P", make_const(1)), make_atom("P", make_const(2)), make_atom("P", make_const(3)), \
        make_atom("Q", make_const(3)), make_atom("Q", make_const(2))]):
    print_formula(f) # should be R(2) and R(3)
print("=====")
# Rule: P(x) ^ Q(y) ^ R(x, y) => T(x, y)
# Facts: P(1), P(2), P(3), Q(3), Q(2), R(3, 2)
for f in apply_rule( 
        make_or(
            make_neg(make_atom("P", make_var("x"))),
            make_neg(make_atom("Q", make_var("y"))),
            make_neg(make_atom("R", make_var("x"), make_var("y"))),
            make_atom("T", make_var("x"), make_var("y"))),
        [make_atom("P", make_const(1)), make_atom("P", make_const(2)), make_atom("P", make_const(3)), \
        make_atom("Q", make_const(3)), make_atom("Q", make_const(2)), make_atom("R", make_const(3), make_const(2))]):
    print_formula(f) # should be T(3, 2)

Q(1)
=====
R(2)
R(3)
=====
T(3, 2)


In [19]:
def forward_chaining(kb, theorem, verbose=True):
    # Salvam baza de date originala, lucram cu o copie
    local_kb = deepcopy(kb)
    # Doua variabile care descriu starea cautarii
    got_new_facts = True   # s-au gasit fapte noi la ultima cuautare
    is_proved = False      # a fost demostrata teorema
    # Verificam daca teorema este deja demonstata
    for fact in filter(is_fact, local_kb):
        if unify(fact, theorem):
            if verbose:
                print("This already in KB: ", end = "")
                print_formula(fact)
            is_proved = True
            break
    while (not is_proved) and got_new_facts:
        got_new_facts = False
        for rule in filter(is_rule, local_kb):
            # Pentru fiecare regula
            new_facts = apply_rule(rule, list(filter(is_fact, local_kb)))
            new_facts = list(filter(lambda fact: not any(list(filter(lambda orig: is_equal_to(fact, orig), local_kb))), new_facts))
            if new_facts:
                if verbose:
                    print("Applied rule: ", end = "")
                    print_formula(rule)
                got_new_facts = True
                for fact in new_facts:
                    #if verbose:
                    #    print("New fact: ", end = "")
                    #    print_formula(fact)
                    if unify(fact, theorem) != False:
                        is_proved = True
                        add_statement(local_kb, fact)
                        if verbose:
                            print("Now in KB: ", end = "")
                            print_formula(fact)
                        break
                    add_statement(local_kb, fact)
            if is_proved:
                break
    if verbose:
        if is_proved:
            print("The theorem is TRUE!")
        else:
            print("The theorem is FALSE!")
    return is_proved

In [20]:
def test_result(result, truth):
    print("Test OK!") if result == truth else print("Test FAILED!")

test_kb = skb
print("================== 0")
test_result(forward_chaining(deepcopy(test_kb), make_atom("Frumos", make_var("x")), True), True)
print("================== 1")
test_result(forward_chaining(deepcopy(test_kb), make_atom("Ploua", make_var("x")), True), True)
print("================== 2")
test_result(forward_chaining(deepcopy(test_kb), make_atom("Ploua", make_const("Joi")), True), True)
print("================== 3")
test_result(forward_chaining(deepcopy(test_kb), make_atom("Frumos", make_const("Sambata")), True), True)
print("================== 4")
test_result(forward_chaining(deepcopy(test_kb),
                             make_atom("Activitate",
                                       make_const("Nectarie"), make_var("sport"), make_const("Sambata")), True), True)

================== 0
This already in KB: Frumos(Luni)
The theorem is TRUE!
Test OK!
================== 1
This already in KB: Ploua(Vineri)
The theorem is TRUE!
Test OK!
================== 2
Applied rule: (SAU NOTConsecutive(?v16, ?v17) NOTConsecutive(?v17, ?v18) NOTConsecutive(?v18, ?v19) NOTFrumos(?v16) NOTFrumos(?v17) NOTFrumos(?v18) Ploua(?v19))
Added statement Ploua(Joi)
Now in KB: Ploua(Joi)
The theorem is TRUE!
Test OK!
================== 3
Applied rule: (SAU NOTConsecutive(?v16, ?v17) NOTConsecutive(?v17, ?v18) NOTConsecutive(?v18, ?v19) NOTFrumos(?v16) NOTFrumos(?v17) NOTFrumos(?v18) Ploua(?v19))
Added statement Ploua(Joi)
Applied rule: (SAU NOTConsecutive(?v7, ?v8) NOTConsecutive(?v8, ?v9) NOTPloua(?v7) NOTPloua(?v8) Frumos(?v9))
Added statement Frumos(Sambata)
Now in KB: Frumos(Sambata)
The theorem is TRUE!
Test OK!
================== 4
Applied rule: (SAU NOTConsecutive(?v16, ?v17) NOTConsecutive(?v17, ?v18) NOTConsecutive(?v18, ?v19) NOTFrumos(?v16) NOTFrumos(?v17) NOTFrumos